In [26]:
import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3) # blocks the calling thread, releases the GIL allowing other python threads to run
    return 42

In [27]:
def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-7 (spin), initial)>
Answer: 42  


In [23]:
# from threads to processes (note the synchronize, and Process)
# NOTE on the cell below this had to change builtin multiprocessing -> multiprocess
# multiprocess was pip installed for Jupyter notebook
from multiprocess import Process, Event
from multiprocess import synchronize

def spin(msg: str, done: synchronize.Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result
    
def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

spinner object: <Process name='Process-4' parent=13497 initial>
Answer: 42  


In [8]:
"""
Jupyter notebooks don't work with multiprocessing because the module 
pickles (serialises) data to send to processes. multiprocess is a fork 
of multiprocessing that uses dill instead of pickle to serialise data which 
allows it to work from within Jupyter notebooks, the API is identical
"""

"\nJupyter notebooks don't work with multiprocessing because the module \npickles (serialises) data to send to processes. multiprocess is a fork \nof multiprocessing that uses dill instead of pickle to serialise data which \nallows it to work from within Jupyter notebooks, the API is identical\n"

In [29]:
import asyncio
import itertools
# allows us to nest event loops (along with nest_asyncio.apply())
# this makes it so I can run examples in Ju
import nest_asyncio

nest_asyncio.apply()

async def supervisor() -> int:
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()
    return result

async def spin(msg: str) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, flush=True, end='')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
        blanks = ' ' * len(status)
        print(f'\r{blanks}\r', end='')

async def slow() -> int:
    await asyncio.sleep(3)
    return 42

def main() -> None:
    #loop = asyncio.get_event_loop()
    #print(loop)
    result = asyncio.run(supervisor())
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

spinner object: <Task pending name='Task-10' coro=<spin() running at /var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_13497/1935890947.py:16>>
| thinking! Answer: 42


In [34]:
import math

PRIME_FIXTURE = [
    (2, True),
    (142702110479723, True),
    (299593572317531, True),
    (3333333333333301, True),
    (3333333333333333, False),
    (3333335652092209, False),
    (4444444444444423, True),
    (4444444444444444, False),
    (4444444488888889, False),
    (5555553133149889, False),
    (5555555555555503, True),
    (5555555555555555, False),
    (6666666666666666, False),
    (6666666666666719, True),
    (6666667141414921, False),
    (7777777536340681, False),
    (7777777777777753, True),
    (7777777777777777, False),
    (9999999999999917, True),
    (9999999999999999, False),
]

NUMBERS = [n for n, _ in PRIME_FIXTURE]

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

In [33]:
is_prime(5_000_111_000_222_021)

True

In [35]:
from time import perf_counter
from typing import NamedTuple

class Result(NamedTuple):
    prime: bool
    elapsed: float

def check(n: int) -> Result:
    t0 = perf_counter()
    prime = is_prime(n)
    return Result(prime, perf_counter() - t0)

def main() -> None:
    print(f'Checking {len(NUMBERS)} numbers sequentially:')
    t0 = perf_counter()
    for n in NUMBERS:
        prime, elapsed = check(n)
        label = 'P' if prime else ' '
        print(f'{n:16} {label} {elapsed:9.6f}s')
    elapsed = perf_counter() - t0
    print(f'Total time: {elapsed:.2f}s')

if __name__ == '__main__':
    main()

Checking 20 numbers sequentially:
               2 P  0.000001s
 142702110479723 P  0.282552s
 299593572317531 P  0.363555s
3333333333333301 P  1.167125s
3333333333333333    0.000006s
3333335652092209    1.164935s
4444444444444423 P  1.484296s
4444444444444444    0.000002s
4444444488888889    1.388999s
5555553133149889    1.494647s
5555555555555503 P  1.559277s
5555555555555555    0.000007s
6666666666666666    0.000000s
6666666666666719 P  1.703077s
6666667141414921    1.635806s
7777777536340681    1.762944s
7777777777777753 P  1.935968s
7777777777777777    0.000006s
9999999999999917 P  2.091291s
9999999999999999    0.000005s
Total time: 18.04s


In [36]:
import sys
from time import perf_counter
from typing import NamedTuple
# AS MENTIONED ABOVE!
# from multiprocessing import Process, SimpleQueue, cpu_count
# from multiprocessing import queues
from multiprocess import Process, SimpleQueue, cpu_count
from multiprocess import queues

class PrimeResult(NamedTuple):
    n: int
    prime: bool
    elapsed: float

JobQueue = queues.SimpleQueue[int]
ResultQueue = queues.SimpleQueue[PrimeResult]

def check(n: int) -> PrimeResult:
    t0 = perf_counter()
    res = is_prime(n)
    return PrimeResult(n, res, perf_counter() - t0)

def worker(jobs: JobQueue, results: ResultQueue) -> None:
    while n := jobs.get():
        results.put(check(n))
    results.put(PrimeResult(0, False, 0.0))

def start_jobs(procs: int, jobs: JobQueue, results: ResultQueue) -> None:
    for n in NUMBERS:
        jobs.put(n)
    for _ in range(procs):
        proc = Process(target=worker, args=(jobs, results))
        proc.start()
        jobs.put(0)

In [45]:
def main() -> None:
   # if len(sys.argv) < 2:
    procs = cpu_count()
    #else:
    #    procs = int(sys.argv[1])
    print(f'Checking {len(NUMBERS)} number with {procs} processes:')
    t0 = perf_counter()
    jobs: JobQueue = SimpleQueue()
    results: ResultQueue = SimpleQueue()
    start_jobs(procs, jobs, results)
    checked = report(procs, results)
    elapsed = perf_counter() - t0
    print(f'{checked} checks in {elapsed:.2f}s')

def report(procs: int, results: ResultQueue) -> int:
    checked = 0
    procs_done = 0
    while procs_done < procs: # having this loop True and breaking when jobs.empty() causes a race condition
        n, prime, elapsed = results.get()
        if n == 0:
            procs_done += 1
        else:
            checked += 1
            label = 'P' if prime else ' '
            print(f'{n:16} {label} {elapsed:9.6f}s')
    return checked

if __name__ == '__main__':
    main()

Checking 20 number with 10 processes:
               2 P  0.000006s
3333333333333333    0.000029s
4444444444444444    0.000006s
5555555555555555    0.000030s
6666666666666666    0.000002s
 142702110479723 P  0.375869s
 299593572317531 P  0.525074s
3333335652092209    1.608493s
7777777777777777    0.000010s
3333333333333301 P  1.736931s
9999999999999999    0.000008s
4444444488888889    1.877297s
4444444444444423 P  1.928638s
5555555555555503 P  2.044051s
5555553133149889    2.060110s
6666666666666719 P  2.198597s
6666667141414921    2.237659s
7777777536340681    2.273445s
7777777777777753 P  2.199313s
9999999999999917 P  2.203441s
20 checks in 3.85s
